[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emiletimothy/Caltech-CS155-2023/blob/main/set5/set5_prob3.ipynb)


## Set 5
## 3. Word2Vec **Principles**

#### Preparation

In [ ]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/P3CHelpers.py

--2023-03-01 11:50:44--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0s      

2023-03-01 11:50:44 (51.0 MB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [ ]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/data/dr_seuss.txt

--2023-03-01 11:50:44--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2023-03-01 11:50:44 (60.9 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [ ]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim

#### Problem D: 
Fill in the generate_traindata and find_most_similar_pairs functions.

In [ ]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2 
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    trainX = []
    trainY = []
    for i in range(len(word_list)):
      Li = get_word_repr(word_to_index, word_list[i])
      s1 = max(i - window_size, 0)
      s2 = min(i + window_size + 1, len(word_list))
      for j in range(s1, s2):
        if (i != j):
          trainX.append(Li)
          Lj = get_word_repr(word_to_index, word_list[j])
          trainY.append(Lj)
    return np.array(trainX), np.array(trainY)

In [ ]:
from traitlets.traitlets import Float
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    # vocab_size = number of unique words in our text file. Will be useful 
    # when adding layers to your neural network
    vocab_size = len(word_to_index)
    
    #Model
    train_dataset = torch.from_numpy(np.array([trainX, trainY])).type(torch.FloatTensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    model = nn.Sequential(
        nn.Linear(vocab_size, 10), 
        nn.Linear(10, vocab_size), nn.Softmax()
        )
    loss_f = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 0.08)
    
    model.train()
    for epoch in range(10):
      for batch_idx, (data, target) in enumerate(train_loader):
          # Erase accumulated gradients
          optimizer.zero_grad()

          # Forward pass
          output = model(data)

          # Calculate loss
          loss = loss_f(output, target)

          # Backward pass
          loss.backward()
          
          # Weight update
          optimizer.step()

      # Track loss each epoch
      print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))
    model.eval()

    print()
    for par in model.named_parameters():
      print("Name: ", par[0], ". Shape: ", par[1].shape)
    # set weights variable below
    weights = model.get_parameter('1.weight').detach().numpy()
    
    print()
    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [ ]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words
Train Epoch: 1  Loss: 5.7302
Train Epoch: 2  Loss: 5.7298
Train Epoch: 3  Loss: 5.7289
Train Epoch: 4  Loss: 5.7276
Train Epoch: 5  Loss: 5.7253
Train Epoch: 6  Loss: 5.7215
Train Epoch: 7  Loss: 5.7163
Train Epoch: 8  Loss: 5.7100
Train Epoch: 9  Loss: 5.7031
Train Epoch: 10  Loss: 5.6961

Name:  0.weight . Shape:  torch.Size([10, 308])
Name:  0.bias . Shape:  torch.Size([10])
Name:  1.weight . Shape:  torch.Size([308, 10])
Name:  1.bias . Shape:  torch.Size([308])

Pair(fear, take), Similarity: 0.9954883
Pair(take, fear), Similarity: 0.9954883
Pair(feet, low), Similarity: 0.99465114
Pair(low, feet), Similarity: 0.99465114
Pair(called, zeep), Similarity: 0.98782295
Pair(zeep, called), Similarity: 0.98782295
Pair(head, wave), Similarity: 0.98776114
Pair(wave, head), Similarity: 0.98776114
Pair(made, only), Similarity: 0.9875726
Pair(only, made), Similarity: 0.9875726
Pair(little, took), Similarity: 0.9871121
Pair(took, little), Similarity: 0.9871121
P

In [ ]:
print()